In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import imageio.v3 as imageio
from dataclasses import dataclass
from IPython.display import clear_output
from render.CollectFrames import collect_saved_frames
matplotlib.use('Agg')

#### Functions to calculate image for the Mandelbrot and Julia sets

In [ ]:
class ComplexSetData:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.c_complex = None
        self.z_complex = None

    def mandelbrot_init(self, min_coords: tuple, max_coords: tuple):
        # Initialize the complex constants c from the viewport
        real_axis = np.linspace(min_coords[0], max_coords[0], self.width)
        imag_axis = np.linspace(max_coords[1], min_coords[1], self.height)
        c_real, c_imag = np.meshgrid(real_axis, imag_axis)
        self.c_complex = c_real + 1j * c_imag
        # Initialize the complex variables z to zeroes
        self.z_complex = np.zeros((self.height, self.width), dtype=np.complex128)
        return self

    def mandelbrot_update(self, active: np.ndarray):
        # Apply Mandelbrot iteration zᵢ₊₁ = zᵢ² + c
        self.z_complex[active] = self.z_complex[active] ** 2 + self.c_complex[active]

    def julia_init(self, min_coords: tuple, max_coords: tuple):
        # Initialize the complex variables z from the viewport
        real_axis = np.linspace(min_coords[0], max_coords[0], self.width)
        imag_axis = np.linspace(max_coords[1], min_coords[1], self.height)
        z_real, z_imag = np.meshgrid(real_axis, imag_axis)
        self.z_complex = z_real + 1j * z_imag
        return self

    def julia_update(self, active: np.ndarray, c: complex):
        # Apply the Julia iteration zᵢ₊₁ = zᵢ² + c 
        self.z_complex[active] = self.z_complex[active] ** 2 + c

In [127]:
def complex_set(set_data: ComplexSetData, set_type: str, num_iter: int, c: complex = None):
    # Initialize the output image and set of active pixels
    image = np.full((set_data.height, set_data.width), num_iter, dtype=int)
    active = np.full((set_data.height, set_data.width), True)
    for iteration in range(num_iter):
        # Update the complex z based on set type
        if set_type.lower() == "mandelbrot":
            set_data.mandelbrot_update(active)
        else:
            set_data.julia_update(active, c)
        # Check for points that escaped (|z| > 2)
        escaped_mask = np.absolute(set_data.z_complex) > 2.0
        # Record the iteration number for escaped points
        image[active & escaped_mask] = iteration
        active &= ~escaped_mask
        # Exit early if all magnitudes have escaped
        if not active.any():
            break
    return image

#### Generator to calculate zooming for the image

In [128]:
@dataclass
class ZoomData:
    point: tuple[float, float]
    num_iter: int
    zoom_factor: float
    iter_kfactor: float
    initial_min_coords: tuple[float, float]
    initial_max_coords: tuple[float, float]

In [129]:
def zoom_complex_set(
        set_data: ComplexSetData,
        set_type: str,
        zoom: ZoomData,
        set_num_iter: int,
        c: complex = None):
    target_real, target_imag = zoom.point
    min_coords = zoom.initial_min_coords
    max_coords = zoom.initial_max_coords
    for _ in range(zoom.num_iter):
        # Calculate the current width of the view W_curr = max_real - min_real
        current_width_coord = max_coords[0] - min_coords[0]
        # Dynamically increase iterations based on zoom level I_calc = I_base + k ⋅ |log(W_curr)|
        current_iter = set_num_iter
        if current_width_coord > 1e-16:
            calculated_iter = set_num_iter + zoom.iter_kfactor * abs(np.log(current_width_coord))
            current_iter = max(set_num_iter, int(calculated_iter))
        # Initialize the complex set grid data based on set type
        if set_type.lower() == "mandelbrot":
            set_data.mandelbrot_init(min_coords, max_coords)
        else:
            set_data.julia_init(min_coords, max_coords)
        # Generate the fractal image for the current view
        image = complex_set(set_data, set_type, current_iter, c)
        yield image, min_coords, max_coords
        # Calculate the dimensions for the next zoom level
        next_width_coord = current_width_coord / zoom.zoom_factor
        next_height_coord = (max_coords[1] - min_coords[1]) / zoom.zoom_factor
        # Calculate the coordinates for the next zoom iteration, centered around the target point
        min_coords = (target_real - next_width_coord / 2, target_imag - next_height_coord / 2)
        max_coords = (target_real + next_width_coord / 2, target_imag + next_height_coord / 2)

#### Function to adjust generalized zooming coordinates for aspect ratio

In [130]:
def adjust_coords_for_aspect(width: int, height: int, zoom_data: ZoomData):
    pixel_aspect_ratio = width / height
    initial_min_real, initial_min_imag = zoom_data.initial_min_coords
    initial_max_real, initial_max_imag = zoom_data.initial_max_coords
    # Calculate the current width and height of the view in the complex plane
    coord_width = initial_max_real - initial_min_real
    coord_height = initial_max_imag - initial_min_imag
    # Calculate the aspect ratio of the current complex coordinate window
    coord_aspect_ratio = coord_width / coord_height if abs(coord_height) > 1e-9 else np.inf
    # If the coordinate aspect differs from the pixel aspect adjustment is needed
    if abs(coord_aspect_ratio - pixel_aspect_ratio) > 1e-6:
        # Calculate the required height in the complex plane
        coord_height_needed = coord_width / pixel_aspect_ratio
        # Calculate of the current view for imaginary aspect
        center_imag = (initial_max_imag + initial_min_imag) / 2
        # Calculate the new center adjusted min and max imaginary coordinates
        new_min_imag = center_imag - coord_height_needed / 2
        new_max_imag = center_imag + coord_height_needed / 2
        # Update coordinates with the new imaginary min and max
        zoom_data.initial_min_coords = (initial_min_real, new_min_imag)
        zoom_data.initial_max_coords = (initial_max_real, new_max_imag)
    return zoom_data

#### Function to visualize the zoomed frames

In [131]:
def create_fractal_frame(
        image: np.ndarray,
        min_coords: tuple,
        max_coords: tuple,
        initial_iter: int,
        kfactor: float,
        dpi: int):
    current_width_coord = max_coords[0] - min_coords[0]
    current_iter_for_frame = initial_iter
    if current_width_coord > 1e-16:
        calculated_iter = initial_iter + kfactor * abs(np.log(current_width_coord))
        current_iter_for_frame = max(initial_iter, int(calculated_iter))
    img_height, img_width = image.shape[:2]
    fig_width_inches = img_width / dpi
    fig_height_inches = img_height / dpi
    frame_fig, frame_ax = plt.subplots(figsize=(fig_width_inches, fig_height_inches), dpi=dpi)
    extent = [min_coords[0], max_coords[0], min_coords[1], max_coords[1]]
    frame_ax.imshow(image, cmap='plasma', extent=extent, vmin=0, vmax=current_iter_for_frame)
    frame_ax.axis('off')
    frame_fig.tight_layout(pad=0)
    frame_fig.canvas.draw()
    frame_image_buf = frame_fig.canvas.buffer_rgba()
    buf_width, buf_height = frame_fig.canvas.get_width_height()
    frame_image = np.frombuffer(frame_image_buf, dtype=np.uint8)
    frame_image = frame_image.reshape(buf_height, buf_width, 4)
    plt.close(frame_fig)
    return frame_image

#### Calculate the Mandelbrot set and collect frames

In [133]:
width = 500
height = 500
initial_iter = 150
fractal_data = ComplexSetData(width=width, height=height)
zoom_data = adjust_coords_for_aspect(width, height, ZoomData(
    point = (-0.743643886906, 0.131825904000002),
    num_iter = 600,
    zoom_factor = 1.05,
    iter_kfactor = 50.0,
    initial_min_coords = (-2.0, -1.5),
    initial_max_coords = (1.0, 1.5)
))

In [ ]:
mandelbrot_folder_path = "gifs/mandelbrot/"
os.makedirs(mandelbrot_folder_path, exist_ok=True)

In [135]:
for idx, (image, curr_min, curr_max) in enumerate(zoom_complex_set(fractal_data, "mandelbrot", zoom_data, initial_iter)):
    print(f"Mandelbrot set frame: {idx}")
    frame = create_fractal_frame(image, curr_min, curr_max, initial_iter, zoom_data.iter_kfactor, 90)
    imageio.imwrite(os.path.join(mandelbrot_folder_path, f"MandelbrotSet{idx:08d}.png"), frame)
clear_output()

In [136]:
mandelbrot_gif_filename = "MandelbrotSet.gif"
collect_saved_frames(mandelbrot_folder_path, mandelbrot_gif_filename)

#### Visualize the Mandelbrot set

![Mandelbrot Set](gifs/mandelbrot/MandelbrotSet.gif)

#### Visualize the Julia set

In [138]:
width = 500
height = 500
initial_iter = 150
julia_c = -0.8 + 0.156j
fractal_data = ComplexSetData(width=width, height=height)
zoom_data = adjust_coords_for_aspect(width, height, ZoomData(
    point = (0.29, 0.210),
    num_iter = 600,
    zoom_factor = 1.05,
    iter_kfactor = 50.0,
    initial_min_coords = (-1.5, -1.5),
    initial_max_coords = (1.5, 1.5)
))

In [ ]:
julia_folder_path = "gifs/julia/"
os.makedirs(julia_folder_path, exist_ok=True)

In [140]:
for idx, (image, curr_min, curr_max) in enumerate(zoom_complex_set(fractal_data, "julia", zoom_data, initial_iter, julia_c)):
    print(f"Julia set frame: {idx}")
    frame = create_fractal_frame(image, curr_min, curr_max, initial_iter, zoom_data.iter_kfactor, 90)
    imageio.imwrite(os.path.join(julia_folder_path, f"JuliaSet{idx:08d}.png"), frame)
clear_output()

In [141]:
julia_gif_filename = "JuliaSet.gif"
collect_saved_frames(julia_folder_path, julia_gif_filename)

#### Visualize the Julia set

![Julia Set](gifs/julia/JuliaSet.gif)